# 220823 - Kevin

## 추천시스템(Recommender System)

* 추천 시스템은 크게 두 가지로 구분 가능
    * 컨텐츠 기반 필터링 (Content-based filtering) : 사용자의 이전 행동, 명시적인 피드백을 통해서 사용자가 좋아하는 것을 추측하고 이를 기반으로 추천
    * 협업 필터링 (Collaborative filtering) : 사용자와 항목 간의 유사성을 동시에 활용하여 추천
* 두 가지를 조합한 Hybrid 방식도 가능

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* Scikit-learn과 유사한 사용 방법

In [62]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


In [63]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

User / Movie / Rating

In [64]:
model = SVD()

In [65]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9332  0.9439  0.9364  0.9327  0.9348  0.9362  0.0041  
MAE (testset)     0.7344  0.7451  0.7393  0.7347  0.7383  0.7384  0.0039  
Fit time          5.64    4.93    4.91    4.89    4.89    5.05    0.29    
Test time         0.16    0.20    0.15    0.19    0.14    0.17    0.02    


{'test_rmse': array([0.9331648 , 0.94391911, 0.93644516, 0.93271199, 0.93479393]),
 'test_mae': array([0.73442942, 0.74508216, 0.73933997, 0.7346605 , 0.73834953]),
 'fit_time': (5.6431028842926025,
  4.928813695907593,
  4.911855697631836,
  4.8929524421691895,
  4.894904375076294),
 'test_time': (0.1635582447052002,
  0.19647598266601562,
  0.15059804916381836,
  0.18749547004699707,
  0.14062213897705078)}

## 컨텐츠 기반 필터링 (Contant-based Filtering)
* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
    * ex) 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천 (왓챠나 넷플릿스 같은)
* 유사도를 기반으로 추천
* 컨텐츠 기반 다음과 같은 장단점이 있다.
    * 장점
        * 많은 수의 사용자를 대상으로 쉽게 확장 가능
        * 사용자가 관심을 갖지 않던 상품 추천 가능
    * 단점
        * 입력 특성을 직접 설계 해야하기 때문에 많은 도메인 지식이 필요
        * 사용자의 기존 관심사항을 기반으로 추천가능

In [66]:
import numpy as np
from surprise import Dataset

In [67]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [68]:
print(raw_data[:, 0])
print(raw_data[:, 1])

[196 186  22 ... 276  13  12]
[ 242  302  377 ... 1090  225  203]


In [69]:
"""해당 데이터가 0부터 시작하도록 하기 위함"""
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [70]:
print(raw_data[:, 0])
print(raw_data[:, 1])

[195 185  21 ... 275  12  11]
[ 241  301  376 ... 1089  224  202]


In [71]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
"""아까 전체 -1을 해줬으므로 shape 출력시 + 1"""
shape = (n_users + 1, n_movies + 1)
shape


(943, 1682)

In [72]:
"""인접 행렬 만들기"""
adj_matrix = np.ndarray(shape, dtype = int)

""""""
for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = 1
adj_matrix
"""1이 있는 위치가 데이터가 있다는 의미"""

'1이 있는 위치가 데이터가 있다는 의미'

In [73]:
my_id, my_vector = 0, adj_matrix[0]
"""내 아이디 전송"""
best_match, best_match_id, best_match_vector = -1, -1, []

"""인접행렬 조사"""
for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id: #유사성 비교
        similarity = np.dot(my_vector, user_vector) #mnp.dot은 array 곱셈 함수
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
            
            
print("Best Match:{}, Best Match ID: {}".format(best_match, best_match_id))

"""이런 식으로 인접행렬을 통해서 유사도를 구하고, 유사도에 따라서 가장 유사한 것을 Return"""

Best Match:183, Best Match ID: 275


'이런 식으로 인접행렬을 통해서 유사도를 구하고, 유사도에 따라서 가장 유사한 것을 Return'

In [74]:
"""추천 리스트 가져오기"""

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
        """내가 보지는 않았으면서, best_match_vector 즉, 275번이 봤던 영화를 Recommed 해준다."""
        
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

### 유클리드 거리를 사용해 추천

$$ euclidean = \sqrt{\sum_{d=1}^D (A_i - B_i)^2} $$
* 거리가 가까울수록(값이 작을수록) 나와 유사한 사용자

In [75]:
my_id, my_vector = 0, adj_matrix[0]
"""내 아이디 전송"""
best_match, best_match_id, best_match_vector = 9999, -1, []

"""인접행렬 조사"""
for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id: #유사성 비교
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
            
            
            
print("Best Match:{}, Best Match ID: {}".format(best_match, best_match_id))

Best Match:14.832396974191326, Best Match ID: 737


### 코사인 유사도를 사용해 추천

$$ cos\theta = \frac{A \cdot B}{||A|| \times ||B||} $$

* 두 벡터가 이루고 있는 각을 계산

In [76]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [78]:
my_id, my_vector = 0, adj_matrix[0]
"""내 아이디 전송"""
best_match, best_match_id, best_match_vector = -1, -1, []

"""인접행렬 조사"""
for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id: #유사성 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
            
print("Best Match:{}, Best Match ID: {}".format(best_match, best_match_id))

Best Match:0.5278586163659506, Best Match ID: 915


In [79]:
"""추천 리스트 가져오기"""

recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)
        """내가 보지는 않았으면서, best_match_vector 즉, 915번이 봤던 영화를 Recommed 해준다."""
        
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


#### 기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [81]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = rating
adj_matrix

"""봤냐 안봤냐를 떠나서 Rating까지 참조해서 추천"""

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [82]:
my_id, my_vector = 0, adj_matrix[0]
"""내 아이디 전송"""
best_match, best_match_id, best_match_vector = 9999, -1, []

"""인접행렬 조사"""
for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id: #유사성 비교
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
            
            
            
print("Best Match:{}, Best Match ID: {}".format(best_match, best_match_id))

Best Match:55.06359959174482, Best Match ID: 737


In [83]:
my_id, my_vector = 0, adj_matrix[0]
"""내 아이디 전송"""
best_match, best_match_id, best_match_vector = -1, -1, []

"""인접행렬 조사"""
for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id: #유사성 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
            
print("Best Match:{}, Best Match ID: {}".format(best_match, best_match_id))

Best Match:0.569065731527988, Best Match ID: 915


## 협업 필터링 (Collabrative Filtering)
* 사용자와 항목의 유사성을 동시에 고려해 ㅊ춴
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
    * 장점
        * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요없다.
        * 기존의 관심사 아니더라도 추천 가능
    * 단점
        * 학습 과정에 나오지 않은 항모은 임베딩을 만들 수 없음
        * 추가 특성을 사용하기 어려움

In [84]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
data = Dataset.load_builtin('ml-100k', prompt=False)

In [85]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9845  0.9744  0.9847  0.9739  0.9789  0.9793  0.0047  
MAE (testset)     0.7756  0.7681  0.7792  0.7696  0.7728  0.7731  0.0040  
Fit time          0.70    0.80    1.03    0.92    0.85    0.86    0.11    
Test time         6.30    6.69    6.83    6.10    4.48    6.08    0.84    


{'test_rmse': array([0.98446528, 0.9744386 , 0.98469001, 0.97393169, 0.97890888]),
 'test_mae': array([0.77557228, 0.76809738, 0.77920929, 0.76960987, 0.77280621]),
 'fit_time': (0.6951150894165039,
  0.7995831966400146,
  1.0285956859588623,
  0.9197750091552734,
  0.85367751121521),
 'test_time': (6.296036243438721,
  6.689013957977295,
  6.832956552505493,
  6.099775552749634,
  4.480147838592529)}

In [86]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9343  0.9266  0.9369  0.9434  0.9345  0.9351  0.0054  
MAE (testset)     0.7374  0.7290  0.7391  0.7415  0.7373  0.7368  0.0042  
Fit time          7.96    8.65    9.41    9.33    7.87    8.64    0.65    
Test time         0.40    0.28    0.31    0.20    0.18    0.28    0.08    


{'test_rmse': array([0.93426872, 0.92658734, 0.93690456, 0.94339429, 0.93445972]),
 'test_mae': array([0.73735595, 0.72895085, 0.73909246, 0.74153291, 0.73725355]),
 'fit_time': (7.955412864685059,
  8.645205974578857,
  9.412377834320068,
  9.32841968536377,
  7.871818780899048),
 'test_time': (0.4044535160064697,
  0.281386137008667,
  0.31068968772888184,
  0.20377469062805176,
  0.17914819717407227)}

In [87]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9582  0.9764  0.9600  0.9673  0.9552  0.9634  0.0076  
MAE (testset)     0.7526  0.7658  0.7545  0.7611  0.7565  0.7581  0.0048  
Fit time          8.76    8.40    7.79    7.74    7.12    7.96    0.57    
Test time         0.25    0.20    0.25    0.18    0.36    0.25    0.06    


{'test_rmse': array([0.9582074 , 0.97635277, 0.96000658, 0.96734341, 0.95515941]),
 'test_mae': array([0.7526109 , 0.76583738, 0.75453869, 0.76106442, 0.75645342]),
 'fit_time': (8.756924152374268,
  8.401148557662964,
  7.792157411575317,
  7.7384865283966064,
  7.120773792266846),
 'test_time': (0.254319429397583,
  0.19946575164794922,
  0.2513265609741211,
  0.17553019523620605,
  0.3595867156982422)}

In [88]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9201  0.9209  0.9174  0.9148  0.9221  0.9191  0.0026  
MAE (testset)     0.7195  0.7225  0.7183  0.7175  0.7232  0.7202  0.0023  
Fit time          362.07  360.97  362.54  361.07  241.76  337.68  47.97   
Test time         5.56    5.72    5.36    5.06    3.54    5.05    0.79    


{'test_rmse': array([0.92007003, 0.92091058, 0.91743405, 0.91478043, 0.92211045]),
 'test_mae': array([0.719547  , 0.72247383, 0.71833019, 0.71747629, 0.72320631]),
 'fit_time': (362.07327699661255,
  360.97121000289917,
  362.54486083984375,
  361.0728635787964,
  241.75621962547302),
 'test_time': (5.558143615722656,
  5.724698305130005,
  5.358666181564331,
  5.057439804077148,
  3.536902666091919)}